# Movies Database: Data Enrichment

by Israel Diaz

## Data Description

The data correspond to the one downloaded from [IMDB source](https://datasets.imdbws.com/).

**IMDb Dataset Details**

Each dataset is contained in a gzipped, tab-separated-values (TSV) formatted file in the UTF-8 character set. The first line in each file contains headers that describe what is in each column. A ‘\N’ is used to denote that a particular field is missing or null for that title/name. The available datasets are as follows:

**title.akas.tsv.gz** - Contains the following information for titles:

* titleId (string) - a tconst, an alphanumeric unique identifier of the title
* ordering (integer) – a number to uniquely identify rows for a given titleId
* title (string) – the localized title
* region (string) - the region for this version of the title
* language (string) - the language of the title
* types (array) - Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning
* attributes (array) - Additional terms to describe this alternative title, not enumerated
* isOriginalTitle (boolean) – 0: not original title; 1: original title

**title.basics.tsv.gz** - Contains the following information for titles:

* tconst (string) - alphanumeric unique identifier of the title
* titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
* primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
* originalTitle (string) - original title, in the original language
* isAdult (boolean) - 0: non-adult title; 1: adult title
* startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
* endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
* runtimeMinutes – primary runtime of the title, in minutes
* genres (string array) – includes up to three genres associated with the title

**title.ratings.tsv.gz** – Contains the IMDb rating and votes information for titles

* tconst (string) - alphanumeric unique identifier of the title
* averageRating – weighted average of all the individual user ratings
* numVotes - number of votes the title has received

## Data Enrichment

The purpose of this section is to add more relevant information to the database. I will add budget and MPAA Rating.
To get this done, I am going to use the TMDB API.

### Load Libraries

In [1]:
## General Libraries
import pandas as pd
import numpy as np
import warnings
import os, time,json

from tqdm.notebook import tqdm_notebook
warnings.simplefilter('ignore')

##Specific
import tmdbsimple as tmdb

## specifying data folder
FOLDER = "data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['basics.csv.gz',
 'aka.csv.gz',
 'ratings.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'mod',
 'backup_data']

### Defining Functions

In [2]:
#@Functions

def write_json(new_data, filename):
    """Appends a list of records (new_data) to a json file (filename).
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""

    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

def get_movie_with_rating(movie_id):
    """ Adapted from source github/celuao/tmdbsimple"""
    #Get the movie object for the current id
    movie = tmdb.Movies(movie_id)

    # Save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()

    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation == US
        if c['iso_3166_1'] == 'US':
            # save a 'certification' key in info with the certification
            info['certification'] = c['certification']

    return info



### Loading API KEY

In [3]:
#@ Api Key
with open('C:/Users/diazi/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key-v3', 'api-key-v4'])

In [4]:
tmdb.API_KEY =  login['api-key-v3']

### Loading basics Dataframe

I will load the previously saved basic dataframe.

In [5]:
# Load in the dataframe:
basics = pd.read_csv(filepath_or_buffer=FOLDER+'basics.csv.gz')

### Years 2000 and 2001

In [6]:
## save years into a variable
YEARS_TO_GET = [2000,2001]
# YEARS_TO_GET = basics['startYear'].drop_duplicates().sort_values(ascending=True)  # Uncomment to get all years' data
YEARS_TO_GET

[2000, 2001]

In [7]:
## declare error variable for further consultations
errors = []

### Retrieve the data from TMDB

In [8]:
#@ Loop over Years and save results into json files

for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    df = basics.loc[basics['startYear']==YEAR].copy()
    movie_ids = df['tconst'].copy()
    previous_df = pd.read_json(JSON_FILE)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    # Iterating over movie ID
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            temp = get_movie_with_rating(movie_id)
            write_json(temp,JSON_FILE)
            time.sleep(0.02)

        except Exception as e:
            errors.append([movie_id, e])

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1432 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1548 [00:00<?, ?it/s]

### Consulting errors

In [10]:
print(f"- Total errors: {len(errors)}")

- Total errors: 462
